In [1]:
import os
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [3]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [4]:
def normalization(data, key1, key2, field, threshold): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    top = data.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= threshold) \
        .drop(col("row_number")) 
        
    df = top.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedData = top.join(df, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedData

In [7]:
LinkinPark = data.groupBy(col("artistId"), col("trackId")).count()

LinkinParkNorm = normalization(LinkinPark, "artistId", "trackId", "count", 1000) \
        .withColumn("id", col("artistId")) \
        .withColumn("id2", col("trackId")) \
        .withColumn("norm_count", col("norm_count")) \
        .select(col("id"), col("id2"), col("norm_count"))     

window = Window.orderBy(col("norm_count").desc())
    
LinkinParkList = LinkinParkNorm.withColumn("position", rank().over(window))\
    .filter(col("position") < 100)\
    .sort(["norm_count","id","id2"],ascending=[0,1,1])\
    .select(col("id"), col("id2")).take(40)

In [8]:
for hit in LinkinParkList:
    print "%s %s" % hit

967993 869415
967998 947428
968004 927380
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
968043 913568
968046 935077
968047 806127
968065 907906
968073 964586
968086 813446
968092 837129
968118 914441
968125 821410
968140 953008
968148 877445
968161 809793
968163 803065
968168 876119
968189 858639
968221 896937
968224 892880
968232 825536
968237 932845
968238 939177
968241 879045
968242 911250
968248 953554
968255 808494
968259 880230
968265 950148
968266 824437
968269 913243
968272 816049
968278 946743
968285 847460
968286 940006
